# Character-level language models

This tutorial shows how to train a character-level language model with a multilayer recurrent neural network. In particular, we will train a multilayer LSTM network that is able to generate text.



## Import necessary package

Import necessary package

In [1]:
import os
import urllib
import zipfile

## Prepare data
We first open the target file and show the first few characters.

In [2]:
# target_file = './data/shediao.txt'
target_file = './data/shediao_11_17.txt'

In [3]:
with open(target_file, 'r') as f:
    print f.read()[0:1000]


　　射雕英雄传

　　

　　第十一回长春服输

　　沙通天见师弟危殆，跃起急格，挡开了梅超风这一抓，两人手腕相交，都感臂酸心惊。这时左边嗤嗤连声，彭连虎的连珠钱镖也已袭到。梅超风顺手把侯通海身子往钱镖上掷去，“啊啾一声大叫，侯通海身上中镖。黄蓉百忙中叫道：“三头蛟，恭喜发财，得了这么多铜钱１沙通天见这一掷势道十分劲急，师弟撞到地下，必受重伤，倏地飞身过去，伸掌在他腰间向上一托。侯通海犹如纸鹞般飞了起来，待得再行落地，那已是自然之势，他一身武功，这般摔一交便毫不相干。只不过左手给这般势道甩了起来，挥拳打出，手臂长短恰到好处，又是重重的打在三个肉瘤之上。
　　梅超风掷人、沙通天救师弟，都只是眨眼间之事，侯通海肉瘤上刚刚中拳，彭连虎的钱镖又已陆续向梅超风打到，同时欧


Then we define a few utility functions to pre-process the dataset.

In [4]:
word_per_line = 64

def read_content_seperate(path):
    max_word_number = word_per_line
    result_string = ''
    with open(path) as ins:   
        for line in ins:
            temp_string = line.decode('utf-8')
            if len(temp_string) < max_word_number:
                result_string = result_string + '\n' + temp_string
            else:
                segment_number = int(len(temp_string)/max_word_number)
                for i in range(segment_number):
                    result_string = result_string + '\n' + temp_string[i*max_word_number: (i+1)*max_word_number]
            
                result_string = result_string + '\n' + temp_string[(i+1)*max_word_number:]
                
    return result_string

def read_content_whole(path):
    with open(path) as ins:        
        return ins.read().decode('utf-8')
    
def read_content(path):
    return read_content_seperate(path)
        
        
# Return a dict which maps each char into an unique int id
def build_vocab(path):
    content = list(read_content(path))
    idx = 1 # 0 is left for zero-padding
    the_vocab = {}
    for word in content:
        if len(word) == 0:
            continue
        if not word in the_vocab:
            the_vocab[word] = idx
            idx += 1
    return the_vocab

# Encode a sentence with int ids
def text2id(sentence, the_vocab):
    words = list(sentence)
    return [the_vocab[w] for w in words if len(w) > 0]
            
# build char vocabluary from input
vocab = build_vocab(target_file)
print('vocab size = %d' %(len(vocab)))

vocab size = 3042


## Create LSTM Model

Now we create the a multi-layer LSTM model. The definition of LSTM cell is implemented in [lstm.py](https://github.com/dmlc/mxnet-notebooks/blob/master/python/tutorials/lstm.py).

In [5]:
import lstm
# Each line contains at most 129 chars. 
seq_len = word_per_line+1
# embedding dimension, which maps a character to a 256-dimension vector
num_embed = 256
# number of lstm layers
num_lstm_layer = 3
# hidden unit in LSTM cell
num_hidden = 512

symbol = lstm.lstm_unroll(
    num_lstm_layer, 
    seq_len,
    len(vocab) + 1,
    num_hidden=num_hidden,
    num_embed=num_embed,
    num_label=len(vocab) + 1, 
    dropout=0.2)


## Train

First, we create a DataIterator

In [6]:
import bucket_io

# The batch size for training
batch_size = 16

# initalize states for LSTM
init_c = [('l%d_init_c'%l, (batch_size, num_hidden)) for l in range(num_lstm_layer)]
init_h = [('l%d_init_h'%l, (batch_size, num_hidden)) for l in range(num_lstm_layer)]
init_states = init_c + init_h

# Even though BucketSentenceIter supports various length examples,
# we simply use the fixed length version here
data_train = bucket_io.BucketSentenceIter(
    target_file, 
    vocab, 
    [seq_len], 
    batch_size,             
    init_states, 
    seperate_char='\n',
    text2id=text2id, 
    read_content=read_content)

Summary of dataset ==================
bucket of len  65 : 2924 samples


Then we can train with the standard `model.fit` approach.

In [7]:
# @@@ AUTOTEST_OUTPUT_IGNORED_CELL
import mxnet as mx
import numpy as np
import logging
logging.getLogger().setLevel(logging.DEBUG)

# We will show a quick demo with only 1 epoch. In practice, we can set it to be 100
num_epoch = 300
# learning rate 
learning_rate = 0.01

# checkpoint_path = './checkpoint/shediao'
checkpoint_path = './checkpoint/shediao_part'

# Evaluation metric
def Perplexity(label, pred):
    loss = 0.
    for i in range(pred.shape[0]):
        loss += -np.log(max(1e-10, pred[i][int(label[i])]))
    return np.exp(loss / label.size)

model = mx.mod.Module(symbol=symbol,
                      data_names=[x[0] for x in data_train.provide_data],
                      label_names=[y[0] for y in data_train.provide_label],
                      context=[mx.gpu(0)])

model.fit(train_data=data_train,
          num_epoch=num_epoch,
          optimizer='sgd',
          optimizer_params={'learning_rate':learning_rate, 'momentum':0, 'wd':0.0001},
          initializer=mx.init.Xavier(factor_type="in", magnitude=2.34),
          eval_metric=mx.metric.np(Perplexity),
          batch_end_callback=mx.callback.Speedometer(batch_size, 20),
          epoch_end_callback=mx.callback.do_checkpoint(checkpoint_path, 5))

INFO:root:Epoch[0] Batch [20]	Speed: 135.29 samples/sec	Perplexity=2162.170146
INFO:root:Epoch[0] Batch [40]	Speed: 134.40 samples/sec	Perplexity=1668.860761
INFO:root:Epoch[0] Batch [60]	Speed: 137.85 samples/sec	Perplexity=1200.667986
INFO:root:Epoch[0] Batch [80]	Speed: 135.70 samples/sec	Perplexity=4337.388763
INFO:root:Epoch[0] Batch [100]	Speed: 135.89 samples/sec	Perplexity=799.584588
INFO:root:Epoch[0] Batch [120]	Speed: 138.65 samples/sec	Perplexity=562.060644
INFO:root:Epoch[0] Batch [140]	Speed: 134.80 samples/sec	Perplexity=465.964565
INFO:root:Epoch[0] Batch [160]	Speed: 132.73 samples/sec	Perplexity=428.486783
INFO:root:Epoch[0] Batch [180]	Speed: 134.85 samples/sec	Perplexity=450.312711
INFO:root:Epoch[0] Train-Perplexity=334.233429
INFO:root:Epoch[0] Time cost=22.128
INFO:root:Epoch[1] Batch [20]	Speed: 135.99 samples/sec	Perplexity=399.842340
INFO:root:Epoch[1] Batch [40]	Speed: 137.06 samples/sec	Perplexity=422.363518
INFO:root:Epoch[1] Batch [60]	Speed: 137.60 sample

INFO:root:Epoch[10] Batch [60]	Speed: 133.13 samples/sec	Perplexity=218.815480
INFO:root:Epoch[10] Batch [80]	Speed: 136.05 samples/sec	Perplexity=251.876378
INFO:root:Epoch[10] Batch [100]	Speed: 136.69 samples/sec	Perplexity=251.621474
INFO:root:Epoch[10] Batch [120]	Speed: 137.73 samples/sec	Perplexity=255.080627
INFO:root:Epoch[10] Batch [140]	Speed: 133.92 samples/sec	Perplexity=229.731658
INFO:root:Epoch[10] Batch [160]	Speed: 135.48 samples/sec	Perplexity=215.455716
INFO:root:Epoch[10] Batch [180]	Speed: 136.81 samples/sec	Perplexity=226.247079
INFO:root:Epoch[10] Train-Perplexity=154.687689
INFO:root:Epoch[10] Time cost=21.475
INFO:root:Epoch[11] Batch [20]	Speed: 137.12 samples/sec	Perplexity=210.673870
INFO:root:Epoch[11] Batch [40]	Speed: 137.28 samples/sec	Perplexity=226.551023
INFO:root:Epoch[11] Batch [60]	Speed: 135.50 samples/sec	Perplexity=211.265385
INFO:root:Epoch[11] Batch [80]	Speed: 132.47 samples/sec	Perplexity=240.708036
INFO:root:Epoch[11] Batch [100]	Speed: 13

INFO:root:Epoch[20] Batch [80]	Speed: 133.32 samples/sec	Perplexity=179.299526
INFO:root:Epoch[20] Batch [100]	Speed: 134.15 samples/sec	Perplexity=178.451834
INFO:root:Epoch[20] Batch [120]	Speed: 135.51 samples/sec	Perplexity=180.834568
INFO:root:Epoch[20] Batch [140]	Speed: 135.00 samples/sec	Perplexity=166.328884
INFO:root:Epoch[20] Batch [160]	Speed: 137.53 samples/sec	Perplexity=156.872062
INFO:root:Epoch[20] Batch [180]	Speed: 138.06 samples/sec	Perplexity=160.015455
INFO:root:Epoch[20] Train-Perplexity=107.894673
INFO:root:Epoch[20] Time cost=21.548
INFO:root:Epoch[21] Batch [20]	Speed: 136.49 samples/sec	Perplexity=152.507144
INFO:root:Epoch[21] Batch [40]	Speed: 136.86 samples/sec	Perplexity=160.988265
INFO:root:Epoch[21] Batch [60]	Speed: 135.66 samples/sec	Perplexity=150.412693
INFO:root:Epoch[21] Batch [80]	Speed: 136.23 samples/sec	Perplexity=174.137713
INFO:root:Epoch[21] Batch [100]	Speed: 135.69 samples/sec	Perplexity=174.111817
INFO:root:Epoch[21] Batch [120]	Speed: 1

INFO:root:Epoch[30] Batch [100]	Speed: 136.78 samples/sec	Perplexity=128.271057
INFO:root:Epoch[30] Batch [120]	Speed: 137.74 samples/sec	Perplexity=131.654851
INFO:root:Epoch[30] Batch [140]	Speed: 134.46 samples/sec	Perplexity=122.318546
INFO:root:Epoch[30] Batch [160]	Speed: 136.11 samples/sec	Perplexity=113.625406
INFO:root:Epoch[30] Batch [180]	Speed: 131.71 samples/sec	Perplexity=116.066102
INFO:root:Epoch[30] Train-Perplexity=78.935585
INFO:root:Epoch[30] Time cost=21.461
INFO:root:Epoch[31] Batch [20]	Speed: 136.08 samples/sec	Perplexity=112.754378
INFO:root:Epoch[31] Batch [40]	Speed: 134.59 samples/sec	Perplexity=118.964622
INFO:root:Epoch[31] Batch [60]	Speed: 137.09 samples/sec	Perplexity=110.879722
INFO:root:Epoch[31] Batch [80]	Speed: 135.13 samples/sec	Perplexity=124.908171
INFO:root:Epoch[31] Batch [100]	Speed: 136.76 samples/sec	Perplexity=124.656820
INFO:root:Epoch[31] Batch [120]	Speed: 138.10 samples/sec	Perplexity=127.585769
INFO:root:Epoch[31] Batch [140]	Speed: 1

INFO:root:Epoch[40] Batch [140]	Speed: 135.91 samples/sec	Perplexity=90.756650
INFO:root:Epoch[40] Batch [160]	Speed: 138.19 samples/sec	Perplexity=85.590317
INFO:root:Epoch[40] Batch [180]	Speed: 136.78 samples/sec	Perplexity=87.029403
INFO:root:Epoch[40] Train-Perplexity=61.319505
INFO:root:Epoch[40] Time cost=21.358
INFO:root:Epoch[41] Batch [20]	Speed: 136.35 samples/sec	Perplexity=84.506714
INFO:root:Epoch[41] Batch [40]	Speed: 137.33 samples/sec	Perplexity=89.883357
INFO:root:Epoch[41] Batch [60]	Speed: 136.73 samples/sec	Perplexity=83.915104
INFO:root:Epoch[41] Batch [80]	Speed: 134.99 samples/sec	Perplexity=94.572494
INFO:root:Epoch[41] Batch [100]	Speed: 137.20 samples/sec	Perplexity=94.076939
INFO:root:Epoch[41] Batch [120]	Speed: 138.24 samples/sec	Perplexity=95.434353
INFO:root:Epoch[41] Batch [140]	Speed: 136.08 samples/sec	Perplexity=87.917393
INFO:root:Epoch[41] Batch [160]	Speed: 136.81 samples/sec	Perplexity=83.028108
INFO:root:Epoch[41] Batch [180]	Speed: 135.47 sampl

...

INFO:root:Epoch[254] Batch [60]	Speed: 101.71 samples/sec	Perplexity=3.751933
INFO:root:Epoch[254] Batch [80]	Speed: 98.43 samples/sec	Perplexity=4.020225
INFO:root:Epoch[254] Batch [100]	Speed: 98.36 samples/sec	Perplexity=3.638975
INFO:root:Epoch[254] Batch [120]	Speed: 103.48 samples/sec	Perplexity=3.703654
INFO:root:Epoch[254] Batch [140]	Speed: 104.93 samples/sec	Perplexity=3.450927
INFO:root:Epoch[254] Batch [160]	Speed: 102.73 samples/sec	Perplexity=3.421646
INFO:root:Epoch[254] Batch [180]	Speed: 104.20 samples/sec	Perplexity=3.577016
INFO:root:Epoch[254] Train-Perplexity=3.990318
INFO:root:Epoch[254] Time cost=28.468
INFO:root:Saved checkpoint to "./checkpoint/shediao_part-0255.params"
INFO:root:Epoch[255] Batch [20]	Speed: 100.75 samples/sec	Perplexity=3.587546
INFO:root:Epoch[255] Batch [40]	Speed: 112.54 samples/sec	Perplexity=3.566221
INFO:root:Epoch[255] Batch [60]	Speed: 134.98 samples/sec	Perplexity=3.767176
INFO:root:Epoch[255] Batch [80]	Speed: 137.86 samples/sec	Perp

INFO:root:Epoch[264] Batch [100]	Speed: 137.42 samples/sec	Perplexity=3.387052
INFO:root:Epoch[264] Batch [120]	Speed: 134.83 samples/sec	Perplexity=3.454314
INFO:root:Epoch[264] Batch [140]	Speed: 137.13 samples/sec	Perplexity=3.435755
INFO:root:Epoch[264] Batch [160]	Speed: 136.04 samples/sec	Perplexity=3.226901
INFO:root:Epoch[264] Batch [180]	Speed: 136.18 samples/sec	Perplexity=3.478800
INFO:root:Epoch[264] Train-Perplexity=3.245876
INFO:root:Epoch[264] Time cost=21.425
INFO:root:Saved checkpoint to "./checkpoint/shediao_part-0265.params"
INFO:root:Epoch[265] Batch [20]	Speed: 138.66 samples/sec	Perplexity=3.484174
INFO:root:Epoch[265] Batch [40]	Speed: 138.11 samples/sec	Perplexity=3.783809
INFO:root:Epoch[265] Batch [60]	Speed: 137.23 samples/sec	Perplexity=3.776184
INFO:root:Epoch[265] Batch [80]	Speed: 133.38 samples/sec	Perplexity=3.769586
INFO:root:Epoch[265] Batch [100]	Speed: 138.17 samples/sec	Perplexity=3.543584
INFO:root:Epoch[265] Batch [120]	Speed: 137.46 samples/sec	

INFO:root:Epoch[274] Batch [140]	Speed: 137.31 samples/sec	Perplexity=3.169024
INFO:root:Epoch[274] Batch [160]	Speed: 135.53 samples/sec	Perplexity=3.155725
INFO:root:Epoch[274] Batch [180]	Speed: 135.28 samples/sec	Perplexity=3.288111
INFO:root:Epoch[274] Train-Perplexity=3.163533
INFO:root:Epoch[274] Time cost=21.470
INFO:root:Saved checkpoint to "./checkpoint/shediao_part-0275.params"
INFO:root:Epoch[275] Batch [20]	Speed: 133.73 samples/sec	Perplexity=3.160969
INFO:root:Epoch[275] Batch [40]	Speed: 135.19 samples/sec	Perplexity=3.217887
INFO:root:Epoch[275] Batch [60]	Speed: 135.49 samples/sec	Perplexity=3.363209
INFO:root:Epoch[275] Batch [80]	Speed: 136.49 samples/sec	Perplexity=3.332137
INFO:root:Epoch[275] Batch [100]	Speed: 135.29 samples/sec	Perplexity=3.245790
INFO:root:Epoch[275] Batch [120]	Speed: 134.18 samples/sec	Perplexity=3.315350
INFO:root:Epoch[275] Batch [140]	Speed: 135.89 samples/sec	Perplexity=3.103540
INFO:root:Epoch[275] Batch [160]	Speed: 136.02 samples/sec	

INFO:root:Epoch[284] Batch [180]	Speed: 135.82 samples/sec	Perplexity=3.073875
INFO:root:Epoch[284] Train-Perplexity=2.900739
INFO:root:Epoch[284] Time cost=21.368
INFO:root:Saved checkpoint to "./checkpoint/shediao_part-0285.params"
INFO:root:Epoch[285] Batch [20]	Speed: 134.81 samples/sec	Perplexity=3.019751
INFO:root:Epoch[285] Batch [40]	Speed: 134.99 samples/sec	Perplexity=3.189451
INFO:root:Epoch[285] Batch [60]	Speed: 135.63 samples/sec	Perplexity=3.202553
INFO:root:Epoch[285] Batch [80]	Speed: 134.75 samples/sec	Perplexity=3.289937
INFO:root:Epoch[285] Batch [100]	Speed: 136.59 samples/sec	Perplexity=3.107184
INFO:root:Epoch[285] Batch [120]	Speed: 136.47 samples/sec	Perplexity=3.152654
INFO:root:Epoch[285] Batch [140]	Speed: 136.88 samples/sec	Perplexity=3.079745
INFO:root:Epoch[285] Batch [160]	Speed: 136.27 samples/sec	Perplexity=2.936281
INFO:root:Epoch[285] Batch [180]	Speed: 137.13 samples/sec	Perplexity=3.040042
INFO:root:Epoch[285] Train-Perplexity=2.701407
INFO:root:Ep

INFO:root:Saved checkpoint to "./checkpoint/shediao_part-0295.params"
INFO:root:Epoch[295] Batch [20]	Speed: 135.35 samples/sec	Perplexity=2.907000
INFO:root:Epoch[295] Batch [40]	Speed: 137.27 samples/sec	Perplexity=3.014757
INFO:root:Epoch[295] Batch [60]	Speed: 136.78 samples/sec	Perplexity=3.066231
INFO:root:Epoch[295] Batch [80]	Speed: 135.50 samples/sec	Perplexity=3.155882
INFO:root:Epoch[295] Batch [100]	Speed: 136.43 samples/sec	Perplexity=2.963087
INFO:root:Epoch[295] Batch [120]	Speed: 135.08 samples/sec	Perplexity=2.911963
INFO:root:Epoch[295] Batch [140]	Speed: 136.20 samples/sec	Perplexity=2.922032
INFO:root:Epoch[295] Batch [160]	Speed: 136.59 samples/sec	Perplexity=2.777879
INFO:root:Epoch[295] Batch [180]	Speed: 138.00 samples/sec	Perplexity=2.923332
INFO:root:Epoch[295] Train-Perplexity=3.073210
INFO:root:Epoch[295] Time cost=21.383
INFO:root:Epoch[296] Batch [20]	Speed: 136.01 samples/sec	Perplexity=2.981348
INFO:root:Epoch[296] Batch [40]	Speed: 134.50 samples/sec	Pe